In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import json
import os
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/why/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class CaptionDataset(Dataset):

    def __init__(self, data_folder, data_name, split):
        
        # Open hdf5 file where images are stored
        self.split = split
        self.train_hf = h5py.File(data_folder + '/train36.hdf5', 'r')
        self.train_features = self.train_hf['image_features']
        self.val_hf = h5py.File(data_folder + '/val36.hdf5', 'r')
        self.val_features = self.val_hf['image_features']

        # Load bottom up image features distribution
        with open(os.path.join(data_folder, self.split + '_GENOME_DETS_' + data_name + '.json'), 'r') as j:
            self.objdet = json.load(j)

        with open(os.path.join(data_folder, self.split + '_TOPIC_CLASS.json'), 'r') as j:
            self.topic = json.load(j)
            
        self.dataset_size = len(self.topic)

    def __getitem__(self, i):
        
        # The Nth caption corresponds to the (N // captions_per_image)th image
        objdet = self.objdet[i]
        
        # Load bottom up image features
        if objdet[0] == "v":
            img = torch.FloatTensor(self.val_features[objdet[1]])
        else:
            img = torch.FloatTensor(self.train_features[objdet[1]])
            
        topic = torch.LongTensor([self.topic[i]])
        
        return img,topic
    
    
    def __len__(self):
        return self.dataset_size


In [5]:
class Net(nn.Module):
    def __init__(self, out_dim = 50, features_dim=2048, dropout=0.3):
        super(Net, self).__init__()
        
        self.fc = nn.Linear(2048, 50)

        self.dropout = nn.Dropout(dropout)
        
        self.linear = nn.Sequential(
            nn.Linear(features_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, out_dim),
        )
        

    def forward(self, x):
        
        batch_size = x.size(0) # one batch
        x = x.mean(1)  # (batch_size, num_pixels, encoder_dim)
        x = self.linear(self.dropout(x))
        
        return F.log_softmax(x)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torchvision

data_folder = '/home/why/image-captioning-bottom-up-top-down/other_dataset'
data_name = 'coco_5_cap_per_img_5_min_word_freq'

# Training settings
batch_size = 128
workers = 1

train_loader = torch.utils.data.DataLoader(
    CaptionDataset(data_folder ,data_name,'TRAIN'),
    batch_size=batch_size, shuffle=True, num_workers=workers)
test_loader = torch.utils.data.DataLoader(
    CaptionDataset(data_folder ,data_name, 'VAL'),
    batch_size=batch_size, shuffle=True, num_workers=workers)



In [5]:
model = Net().to(device)

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target = target.squeeze()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))


def test():
    test_loss = 0
    correct = 0
    for batch_idx,(data, target) in enumerate(test_loader):
        batch_size = data.size(0)
        data, target = data.cuda(), target.cuda()
        data = Variable(data,volatile = True)
        target = Variable(target,volatile = True)
        #target = target.squeeze()
        output = model(data)
        # sum up batch loss
        #loss += F.nll_loss(output, target.squeeze(),size_average=False)
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #correct += pred.eq(target.data.view_as(pred)).sum()
        correct += pred.eq(target).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [7]:
for epoch in range(1, 50):
    
    train(epoch)
    test()

/home/why/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/113287 (0%)]	Loss: 3.904691
Train Epoch: 1 [1280/113287 (1%)]	Loss: 3.891490
Train Epoch: 1 [2560/113287 (2%)]	Loss: 3.919845
Train Epoch: 1 [3840/113287 (3%)]	Loss: 3.900365
Train Epoch: 1 [5120/113287 (5%)]	Loss: 3.865664
Train Epoch: 1 [6400/113287 (6%)]	Loss: 3.863529
Train Epoch: 1 [7680/113287 (7%)]	Loss: 3.852815
Train Epoch: 1 [8960/113287 (8%)]	Loss: 3.853329
Train Epoch: 1 [10240/113287 (9%)]	Loss: 3.849704
Train Epoch: 1 [11520/113287 (10%)]	Loss: 3.848873
Train Epoch: 1 [12800/113287 (11%)]	Loss: 3.843569
Train Epoch: 1 [14080/113287 (12%)]	Loss: 3.822124
Train Epoch: 1 [15360/113287 (14%)]	Loss: 3.799979
Train Epoch: 1 [16640/113287 (15%)]	Loss: 3.812196
Train Epoch: 1 [17920/113287 (16%)]	Loss: 3.775379
Train Epoch: 1 [19200/113287 (17%)]	Loss: 3.771463
Train Epoch: 1 [20480/113287 (18%)]	Loss: 3.784264
Train Epoch: 1 [21760/113287 (19%)]	Loss: 3.765289
Train Epoch: 1 [23040/113287 (20%)]	Loss: 3.787389
Train Epoch: 1 [24320/113287 (21%)]	Loss: 3.744679


/home/why/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/why/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0000, Accuracy: 1975/5000 (39%)

Train Epoch: 2 [0/113287 (0%)]	Loss: 2.815035
Train Epoch: 2 [1280/113287 (1%)]	Loss: 2.872782
Train Epoch: 2 [2560/113287 (2%)]	Loss: 2.681639
Train Epoch: 2 [3840/113287 (3%)]	Loss: 2.781986
Train Epoch: 2 [5120/113287 (5%)]	Loss: 2.642687
Train Epoch: 2 [6400/113287 (6%)]	Loss: 2.847109
Train Epoch: 2 [7680/113287 (7%)]	Loss: 2.717479
Train Epoch: 2 [8960/113287 (8%)]	Loss: 2.744192
Train Epoch: 2 [10240/113287 (9%)]	Loss: 2.672832
Train Epoch: 2 [11520/113287 (10%)]	Loss: 2.626578
Train Epoch: 2 [12800/113287 (11%)]	Loss: 2.498728
Train Epoch: 2 [14080/113287 (12%)]	Loss: 2.650059
Train Epoch: 2 [15360/113287 (14%)]	Loss: 2.519052
Train Epoch: 2 [16640/113287 (15%)]	Loss: 2.563042
Train Epoch: 2 [17920/113287 (16%)]	Loss: 2.580826
Train Epoch: 2 [19200/113287 (17%)]	Loss: 2.449224
Train Epoch: 2 [20480/113287 (18%)]	Loss: 2.593746
Train Epoch: 2 [21760/113287 (19%)]	Loss: 2.551198
Train Epoch: 2 [23040/113287 (20%)]	Loss: 

Train Epoch: 3 [89600/113287 (79%)]	Loss: 1.644827
Train Epoch: 3 [90880/113287 (80%)]	Loss: 1.452450
Train Epoch: 3 [92160/113287 (81%)]	Loss: 1.367846
Train Epoch: 3 [93440/113287 (82%)]	Loss: 1.543216
Train Epoch: 3 [94720/113287 (84%)]	Loss: 1.560789
Train Epoch: 3 [96000/113287 (85%)]	Loss: 1.361784
Train Epoch: 3 [97280/113287 (86%)]	Loss: 1.617597
Train Epoch: 3 [98560/113287 (87%)]	Loss: 1.575597
Train Epoch: 3 [99840/113287 (88%)]	Loss: 1.529496
Train Epoch: 3 [101120/113287 (89%)]	Loss: 1.560580
Train Epoch: 3 [102400/113287 (90%)]	Loss: 1.439574
Train Epoch: 3 [103680/113287 (91%)]	Loss: 1.610497
Train Epoch: 3 [104960/113287 (93%)]	Loss: 1.324591
Train Epoch: 3 [106240/113287 (94%)]	Loss: 1.388606
Train Epoch: 3 [107520/113287 (95%)]	Loss: 1.590441
Train Epoch: 3 [108800/113287 (96%)]	Loss: 1.500149
Train Epoch: 3 [110080/113287 (97%)]	Loss: 1.550954
Train Epoch: 3 [111360/113287 (98%)]	Loss: 1.430126
Train Epoch: 3 [112640/113287 (99%)]	Loss: 1.630909

Test set: Average lo

...

Train Epoch: 42 [58880/113287 (52%)]	Loss: 0.907208
Train Epoch: 42 [60160/113287 (53%)]	Loss: 1.156478
Train Epoch: 42 [61440/113287 (54%)]	Loss: 0.893160
Train Epoch: 42 [62720/113287 (55%)]	Loss: 0.825909
Train Epoch: 42 [64000/113287 (56%)]	Loss: 1.100913
Train Epoch: 42 [65280/113287 (58%)]	Loss: 0.830227
Train Epoch: 42 [66560/113287 (59%)]	Loss: 1.078993
Train Epoch: 42 [67840/113287 (60%)]	Loss: 0.843055
Train Epoch: 42 [69120/113287 (61%)]	Loss: 0.792106
Train Epoch: 42 [70400/113287 (62%)]	Loss: 0.976796
Train Epoch: 42 [71680/113287 (63%)]	Loss: 1.143017
Train Epoch: 42 [72960/113287 (64%)]	Loss: 0.758432
Train Epoch: 42 [74240/113287 (65%)]	Loss: 1.009276
Train Epoch: 42 [75520/113287 (67%)]	Loss: 0.787502
Train Epoch: 42 [76800/113287 (68%)]	Loss: 0.874549
Train Epoch: 42 [78080/113287 (69%)]	Loss: 0.896680
Train Epoch: 42 [79360/113287 (70%)]	Loss: 0.848468
Train Epoch: 42 [80640/113287 (71%)]	Loss: 0.852713
Train Epoch: 42 [81920/113287 (72%)]	Loss: 0.903628
Train Epoch:

Train Epoch: 44 [30720/113287 (27%)]	Loss: 0.917543
Train Epoch: 44 [32000/113287 (28%)]	Loss: 1.279707
Train Epoch: 44 [33280/113287 (29%)]	Loss: 0.978142
Train Epoch: 44 [34560/113287 (30%)]	Loss: 1.222848
Train Epoch: 44 [35840/113287 (32%)]	Loss: 0.922946
Train Epoch: 44 [37120/113287 (33%)]	Loss: 0.991227
Train Epoch: 44 [38400/113287 (34%)]	Loss: 0.760550
Train Epoch: 44 [39680/113287 (35%)]	Loss: 0.958628
Train Epoch: 44 [40960/113287 (36%)]	Loss: 0.957031
Train Epoch: 44 [42240/113287 (37%)]	Loss: 0.950453
Train Epoch: 44 [43520/113287 (38%)]	Loss: 0.917420
Train Epoch: 44 [44800/113287 (40%)]	Loss: 0.837877
Train Epoch: 44 [46080/113287 (41%)]	Loss: 0.802961
Train Epoch: 44 [47360/113287 (42%)]	Loss: 0.890064
Train Epoch: 44 [48640/113287 (43%)]	Loss: 1.106075
Train Epoch: 44 [49920/113287 (44%)]	Loss: 1.050214
Train Epoch: 44 [51200/113287 (45%)]	Loss: 0.867268
Train Epoch: 44 [52480/113287 (46%)]	Loss: 1.058190
Train Epoch: 44 [53760/113287 (47%)]	Loss: 0.803634
Train Epoch:

Train Epoch: 46 [2560/113287 (2%)]	Loss: 1.036689
Train Epoch: 46 [3840/113287 (3%)]	Loss: 0.804140
Train Epoch: 46 [5120/113287 (5%)]	Loss: 0.904875
Train Epoch: 46 [6400/113287 (6%)]	Loss: 0.877936
Train Epoch: 46 [7680/113287 (7%)]	Loss: 1.114337
Train Epoch: 46 [8960/113287 (8%)]	Loss: 0.943369
Train Epoch: 46 [10240/113287 (9%)]	Loss: 0.841765
Train Epoch: 46 [11520/113287 (10%)]	Loss: 0.974125
Train Epoch: 46 [12800/113287 (11%)]	Loss: 0.999456
Train Epoch: 46 [14080/113287 (12%)]	Loss: 0.616406
Train Epoch: 46 [15360/113287 (14%)]	Loss: 1.159494
Train Epoch: 46 [16640/113287 (15%)]	Loss: 0.748967
Train Epoch: 46 [17920/113287 (16%)]	Loss: 0.864580
Train Epoch: 46 [19200/113287 (17%)]	Loss: 0.887319
Train Epoch: 46 [20480/113287 (18%)]	Loss: 1.030105
Train Epoch: 46 [21760/113287 (19%)]	Loss: 0.826412
Train Epoch: 46 [23040/113287 (20%)]	Loss: 0.909408
Train Epoch: 46 [24320/113287 (21%)]	Loss: 0.939883
Train Epoch: 46 [25600/113287 (23%)]	Loss: 1.031458
Train Epoch: 46 [26880/11

Train Epoch: 47 [89600/113287 (79%)]	Loss: 0.839537
Train Epoch: 47 [90880/113287 (80%)]	Loss: 0.966487
Train Epoch: 47 [92160/113287 (81%)]	Loss: 0.961458
Train Epoch: 47 [93440/113287 (82%)]	Loss: 0.977131
Train Epoch: 47 [94720/113287 (84%)]	Loss: 0.850925
Train Epoch: 47 [96000/113287 (85%)]	Loss: 1.072795
Train Epoch: 47 [97280/113287 (86%)]	Loss: 0.885150
Train Epoch: 47 [98560/113287 (87%)]	Loss: 0.962181
Train Epoch: 47 [99840/113287 (88%)]	Loss: 1.041693
Train Epoch: 47 [101120/113287 (89%)]	Loss: 0.997406
Train Epoch: 47 [102400/113287 (90%)]	Loss: 1.243246
Train Epoch: 47 [103680/113287 (91%)]	Loss: 0.828854
Train Epoch: 47 [104960/113287 (93%)]	Loss: 1.003646
Train Epoch: 47 [106240/113287 (94%)]	Loss: 0.963988
Train Epoch: 47 [107520/113287 (95%)]	Loss: 0.831327
Train Epoch: 47 [108800/113287 (96%)]	Loss: 0.890200
Train Epoch: 47 [110080/113287 (97%)]	Loss: 1.098419
Train Epoch: 47 [111360/113287 (98%)]	Loss: 0.979133
Train Epoch: 47 [112640/113287 (99%)]	Loss: 0.910585

T

KeyboardInterrupt: 

In [8]:
torch.save(model, "models/image_topic.pth")

/home/why/anaconda3/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torchvision

data_folder = '/home/why/image-captioning-bottom-up-top-down/other_dataset'
data_name = 'coco_5_cap_per_img_5_min_word_freq'

# Training settings
batch_size = 128
workers = 1

test_loader = torch.utils.data.DataLoader(
    CaptionDataset(data_folder ,data_name, 'TEST'),
    batch_size=batch_size, shuffle=True, num_workers=workers)


In [6]:
model = torch.load("models/image_topic.pth")

In [8]:
for batch_idx,(data, target) in enumerate(test_loader):
    break

In [9]:
data, target = data.cuda(), target.cuda()
data = Variable(data,volatile = True)
target = Variable(target,volatile = True)
output = model(data)


/home/why/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/why/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/why/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
_,ind = output.topk(1,1 ,True,True)

In [17]:
ind

tensor([[ 6],
        [49],
        [38],
        [ 0],
        [49],
        [19],
        [49],
        [15],
        [21],
        [49],
        [36],
        [23],
        [12],
        [ 7],
        [ 7],
        [ 1],
        [20],
        [39],
        [22],
        [16],
        [25],
        [14],
        [20],
        [11],
        [26],
        [25],
        [ 4],
        [ 6],
        [21],
        [47],
        [25],
        [49],
        [ 2],
        [22],
        [24],
        [30],
        [33],
        [23],
        [ 2],
        [20],
        [37],
        [14],
        [16],
        [46],
        [11],
        [20],
        [41],
        [28],
        [18],
        [35],
        [ 9],
        [42],
        [43],
        [22],
        [ 4],
        [20],
        [ 2],
        [38],
        [11],
        [33],
        [46],
        [ 2],
        [43],
        [39],
        [47],
        [ 7],
        [ 2],
        [33],
        [48],
        [25],
        [15],
      

In [ ]:
def test():
    test_loss = 0
    correct = 0
    for batch_idx,(data, target) in enumerate(test_loader):
        batch_size = data.size(0)
        data, target = data.cuda(), target.cuda()
        data = Variable(data,volatile = True)
        target = Variable(target,volatile = True)
        #target = target.squeeze()
        output = model(data)
        # sum up batch loss
        #loss += F.nll_loss(output, target.squeeze(),size_average=False)
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        #correct += pred.eq(target.data.view_as(pred)).sum()
        correct += pred.eq(target).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))